In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [1]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Load pretrained BART model and tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Example input text (default data)
text = "The quick brown fox jumps over the lazy dog. The dog was not amused."

# Tokenize input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Example target summary
target_summary = "A fox jumps over a lazy dog."

# Tokenize target summary
targets = tokenizer(target_summary, return_tensors="pt", padding=True, truncation=True)

# Prepare input IDs and attention masks
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
decoder_input_ids = targets.input_ids
labels = targets.input_ids.clone()
labels[labels == tokenizer.pad_token_id] = -100  # Mask padding tokens for loss calculation

# Forward pass (your required format)
outputs = model(input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels)

# Loss and logits
loss = outputs.loss
logits = outputs.logits

print(f"Loss: {loss.item()}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loss: 10.819597244262695


In [2]:
import torch
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration

# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Load BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenization function
def preprocess_data(example):
    # Tokenize article (input)
    inputs = tokenizer(example['article'], max_length=512, truncation=True, padding="max_length")
    # Tokenize highlights (target summary)
    targets = tokenizer(example['highlights'], max_length=128, truncation=True, padding="max_length")

    inputs['labels'] = targets['input_ids']
    return inputs

# Apply tokenization to a small subset for demo (like 100 samples)
train_data = dataset["train"].select(range(100)).map(preprocess_data, batched=True)

# Convert to torch dataset
class CNNDailyMailDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = torch.tensor(item['input_ids'])
        attention_mask = torch.tensor(item['attention_mask'])
        labels = torch.tensor(item['labels'])

        decoder_input_ids = labels[:-1]
        labels = labels[1:]

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'decoder_input_ids': decoder_input_ids,
            'labels': labels
        }


# Prepare dataloader
train_dataset = CNNDailyMailDataset(train_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training loop (1 epoch demo)
model.train()
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    decoder_input_ids = batch['decoder_input_ids'].to(device)
    labels = batch['labels'].to(device)

    outputs = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=decoder_input_ids,
                    labels=labels)

    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Compute perplexity for the batch
    perplexity = torch.exp(loss).item()

    print(f"Loss: {loss.item():.4f} | Perplexity: {perplexity:.4f}")


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [6]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Load tokenizer and model (can be your fine-tuned checkpoint too)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Example article for summarization
article = """The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Constructed from 1887 to 1889 as the entrance to the 1889 World's Fair, it was initially criticized
by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France
and one of the most recognizable structures in the world."""

# Tokenize the article
inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
summary_ids = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=128,
    num_beams=4,             # Beam search for better summaries
    early_stopping=True
)

# Decode and print the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:\n", summary)


Summary:
 The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.  It was built in 1887.It is named after the engineer Gustave Eifel, whose company designed and built the tower.  The tower was completed in 1891.Constructed from 1887 to 1889 as the entrance to the 1889 World's Fair, it was initially criticized  by some of France's leading artists and intellectuals for its design, but it has become a global cultural icon of France  and one of the most recognizable structures in the world.


In [9]:
!pip install evaluate
!pip install rouge_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=257c205b8a1e986b68f3f6d9116400c14c9a29c7a7b0327ce382137795c88537
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [10]:
import evaluate
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration

# Load model, tokenizer, and validation data
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to("cuda")

dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:100]")  # small subset for demo
rouge = evaluate.load("rouge")

# Function to generate summaries
def generate_summary(article):
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True).to("cuda")
    summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_length=128, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Collect generated summaries and references
generated_summaries = []
reference_summaries = []

for example in dataset:
    generated = generate_summary(example["article"])
    reference = example["highlights"]

    generated_summaries.append(generated)
    reference_summaries.append(reference)

# Compute ROUGE scores
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print(results)


{'rouge1': np.float64(0.2745842864787168), 'rouge2': np.float64(0.10893167542922544), 'rougeL': np.float64(0.18090998263584887), 'rougeLsum': np.float64(0.22056727173543483)}


In [11]:
# Print ROUGE results in a cleaner way
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)

print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")
print(f"ROUGE-Lsum: {results['rougeLsum']:.4f}")


ROUGE-1: 0.2746
ROUGE-2: 0.1089
ROUGE-L: 0.1809
ROUGE-Lsum: 0.2206


In [12]:
# Load the BLEU metric
bleu = evaluate.load("bleu")

# Compute BLEU score
bleu_results = bleu.compute(predictions=generated_summaries, references=[[ref] for ref in reference_summaries])

# Print BLEU score
print(f"BLEU Score: {bleu_results['bleu']:.4f}")

BLEU Score: 0.0532


In [15]:
# import torch
# from transformers import BartTokenizer, BartForConditionalGeneration

# # Load tokenizer and model (can be your fine-tuned checkpoint too)
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# # Use GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)
# model.eval()

# Example article for summarization
article = """Elon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa's administrative capital.[2][3] He is of British and Pennsylvania Dutch ancestry.[4][5] His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa.[6][7][8][a] His father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, emerald dealer, and property developer, who partly owned a rental lodge at Timbavati Private Nature Reserve.[15][16][17][18] Elon has a younger brother, Kimbal, a younger sister, Tosca, and four paternal half-siblings.[19][20][8][21] Musk was raised in the Anglican Church, in which he was baptized.[22][23]"""
# Tokenize the article
inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True).to(device)

# Generate summary
summary_ids = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=128,
    num_beams=4,             # Beam search for better summaries
    early_stopping=True
)

# Decode and print the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:\n", summary)


Summary:
 Elon Reeve Musk was born on June 28, 1971, in Pretoria, South Africa's administrative capital.[2][3] He is of British and Pennsylvania Dutch ancestry.[4][5] His mother, Maye (née Haldeman), is a model and dietitian born in Saskatchewan, Canada, and raised in South Africa.[6][7][8][a] His father, Errol Musk, is a South African electromechanical engineer, pilot, sailor, consultant, emerald dealer, and property developer, who partly owned a rental lodge at Timbavati Private Nature Reserve.[15
